## Sherirto/CSTAG

bash command

```shell
export HF_ENDPOINT=https://hf-mirror.com
huggingface-cli download --repo-type dataset --resume-download Sherirto/CSTAG --local-dir graph-feature-selection/data/CSTAG --local-dir-use-symlinks False
```

In [ ]:
CSTAG_DATASET_BASE_PATH = "../data/CSTAG"
# 查找文件夹下所有目录下的.pt文件，返回文件路径列表
import os
import dgl
import numpy  as np
import pandas as pd
import torch

def find_files(dir_path, end_with='.pt'):
    file_list = []
    for root, dirs, files in os.walk(dir_path):
        for file in files:
            if file.endswith(end_with):
                file_list.append(os.path.join(root, file))
    return file_list

find_files(CSTAG_DATASET_BASE_PATH, '.pt')

In [ ]:
datasets = ["Children", "Computers", "Fitness", "History", "Photo"]
num_data_splits = 10
train_prop = 0.5
valid_prop = 0.25
save_path_base = "../data"

for dataset in datasets:
    # edges
    edges = None
    graph_file = find_files(f"{CSTAG_DATASET_BASE_PATH}/{dataset}", '.pt')
    glist, label_dict = dgl.load_graphs(graph_file[0])
    for i, g in enumerate(glist):
        print(f"Dataset: {dataset}, Graph {i+1}:")
        print(f"Number of nodes: {g.number_of_nodes()}")
        edges = torch.tensor(np.array(g.edges()).T)
        print(f"Number of edges: {g.number_of_edges()}, edges dtype: {edges.dtype}")
    # node labels
    csv_file = find_files(f"{CSTAG_DATASET_BASE_PATH}/{dataset}", '.csv')
    csv_data = pd.read_csv(csv_file[0])
    node_labels = torch.tensor(np.array(csv_data['label'].values))
    print(f"Number of labels: {len(node_labels)}, node_labels dtype: {node_labels.dtype}")
    # node features
    features_file = find_files(f"{CSTAG_DATASET_BASE_PATH}/{dataset}", '.npy')
    node_features = torch.tensor(np.load(features_file[0])).to(torch.float)
    print(f"Number of features: {len(node_features)}, node_features dtype: {node_features.dtype}")
    print(f"Feature dimension: {len(node_features[0])}")
    # Splits
    seed = 0
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    num_nodes = len(node_labels)
    mask_number = torch.zeros(num_data_splits, num_nodes)
    for i in range(num_data_splits):
        mask_number[i] = torch.randperm(num_nodes)
    train_masks = (mask_number<=(train_prop*num_nodes))
    val_masks = (torch.logical_and(mask_number<=((train_prop+valid_prop)*num_nodes),mask_number>(train_prop*num_nodes)))
    test_masks = (mask_number>((train_prop+valid_prop)*num_nodes))
    print(f"mask ratio(train:val:test): {train_masks.sum().item()/num_nodes/num_data_splits:.2f}"\
          f":{val_masks.sum().item()/num_nodes/num_data_splits:.2f}"
            f":{test_masks.sum().item()/num_nodes/num_data_splits:.2f}")
    np.savez(f'{save_path_base}/{dataset}.npz',
            node_features=node_features.numpy(),
            node_labels=node_labels.numpy(),
            edges=edges.numpy(),
            train_masks=train_masks.numpy(),
            val_masks=val_masks.numpy(),
            test_masks=test_masks.numpy())
    print(f"save {dataset} graph, train, val, test masks to {save_path_base}/{dataset}.npz")
    print("-" * 50)

## For roman-empire, amazon-ratings, minesweeper, tolokers, questions datasets , please refer to [CLICK THIS LINK](https://github.com/yandex-research/heterophilous-graphs/tree/main/data) for download